In [1]:
import h5py
import numpy as np
import PIL.Image as Image
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from tensorflow.keras.callbacks import ModelCheckpoint
import numpy as np
import PIL.Image as Image
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
from skimage.metrics import structural_similarity as ssim

In [2]:
from IPython.display import clear_output

import tensorflow as tf
import numpy as np

def load_h5_to_dictionary(file_path):
    data_dict = {}
    with h5py.File(file_path, 'r') as hf:
        for key in hf.keys():
            print(key)
            # Create a list to store the arrays for each key
            array_list = []
            # Get the group corresponding to the current key
            group = hf[key]
            for dataset_name in group.keys():
                # Append each dataset (array) to the array list
                array_list.append(np.array(group[dataset_name]))
            # Store the array list for the current key in the dictionary
            data_dict[key] = array_list
    return data_dict

def find_lowest_dimensions(images_list1, images_list2):
    # Find the lowest height and width across both lists
    lowest_height = min(images_list1[0].shape[0], images_list2[0].shape[0])
    lowest_width = min(images_list1[0].shape[1], images_list2[0].shape[1])
    return lowest_height, lowest_width

def crop_images_to_lowest_dimensions(images_list, lowest_height, lowest_width):
    cropped_images = [image[:lowest_height, :lowest_width,:] for image in images_list]
    return cropped_images

def crop_1d_to_lowest_dimensions(images_list, lowest_height, lowest_width):
    cropped_images = [image[:lowest_height, :lowest_width] for image in images_list]
    return cropped_images

def map_to_r(number):
    colorr = [255, 128, 0, 0, 128, 0, 0, 0, 128, 0, 0, 0, 128, 0, 128, 255, 128, 255, 0, 128]
    if number < 20:
        return int(colorr[number])
    else:
        return 0

def map_to_g(number):
    colorg = [0, 128, 255, 128, 0, 0, 128, 0, 0, 128, 0, 128, 0, 128, 255, 128, 0, 128, 255, 128]
    if number < 20:
        return int(colorg[number])
    else:
        return 0

def map_to_b(number):
    colorb = [0, 0, 0, 128, 128, 255, 255, 128, 0, 128, 128, 255, 128, 0, 0, 0, 128, 255, 255, 128]
    if number < 20:
        return int(colorb[number])
    else:
        return 255

def map_to_desired_structure(input_image, output_image):
    return {'pixel_values': input_image, 'labels': output_image}
def map_to_desired_structure_weather(input_image, output_image, weather):
    return {'pixel_values': input_image, 'labels': output_image, 'weather': weather}

def map_values(value):
    # if value in [1, 2, 6, 7, 8, 16, 21, 255]:
    #     return 1
    # elif value in [11, 12]:
    #     return 2
    # elif value in [13, 14, 15, 17]:
    #     return 3
    # elif value == 10:
    #     return 4
    # elif value == 0:
    #     return 0
    # else:
    #     return 1
    # if value==255:
    #     value= 22
    return value

def generator(input_dict, output_dict, input_indices, input_dic_keys):
    vectorized_map = np.vectorize(map_values)
    for key in input_dic_keys:
        for index in input_indices[key]:
            if index < len(input_dict[key]):
                input_image = input_dict[key][index]
                output_image = output_dict[key][index]
                
                # Apply transformations lazily
                input_image = tf.reverse(input_image, axis=[-1])
                input_image = tf.transpose(input_image, (2, 0, 1))  # Adjust to (channels, height, width)
                
                # Vectorize and map values for the label
                output_image = vectorized_map(output_image)

                # Yield one image and label pair at a time
                yield input_image, output_image

def create_tf_dataset(input_dict, output_dict, input_indices, input_dic_keys, batch_size=5):
    # Use a generator to load the data lazily
    dataset = tf.data.Dataset.from_generator(
        lambda: generator(input_dict, output_dict, input_indices, input_dic_keys),
        output_signature=(
            tf.TensorSpec(shape=(None, None, None), dtype=tf.float32),  # Input image shape
            tf.TensorSpec(shape=(None, None), dtype=tf.float32)          # Output image shape
        )
    )

    # Preprocess dataset by mapping to the desired structure
    dataset = dataset.map(map_to_desired_structure)

    # Shuffle and batch the dataset
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)  # Load batches asynchronously

    return dataset

def generator_pert(input_dict, output_dict, input_indices, input_dic_keys):
    vectorized_map = np.vectorize(map_values)
    # print(input_dict)
    for key in input_dic_keys:
        for key2 in input_indices[key]:
            print(key2)
            for index in input_indices[key][key2]:
                if index < len(input_dict[key][key2]):
                    input_image = input_dict[key][key2][index]
                    output_image = output_dict[key][key2][index]
                    
                    # Apply transformations lazily
                    input_image = tf.reverse(input_image, axis=[-1])
                    input_image = tf.transpose(input_image, (2, 0, 1))  # Adjust to (channels, height, width)
                    
                    # Vectorize and map values for the label
                    output_image = vectorized_map(output_image)

                    # Yield one image and label pair at a time
                    yield input_image, output_image, key2

def create_tf_dataset_pert(input_dict, output_dict, input_indices, input_dic_keys, batch_size=5):
    # Use a generator to load the data lazily
    dataset = tf.data.Dataset.from_generator(
        lambda: generator_pert(input_dict, output_dict, input_indices, input_dic_keys),
        output_signature=(
            tf.TensorSpec(shape=(None, None, None), dtype=tf.float32),  # Input image shape
            tf.TensorSpec(shape=(None, None), dtype=tf.float32),        # Output image shape
            tf.TensorSpec(shape=(), dtype=tf.string)                    # String data shape
        )
    )

    # Preprocess dataset by mapping to the desired structure
    dataset = dataset.map(map_to_desired_structure_weather)

    # Shuffle and batch the dataset
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)  # Load batches asynchronously

    return dataset

def calculate_mse(gt_mask, pred_mask):
    """Calculates the Mean Squared Error between the ground truth mask and predicted mask."""
    return np.mean((gt_mask - pred_mask) ** 2)
def calculate_iou(gt_mask, pred_mask, num_classes):
    """Calculates Intersection over Union (IoU) for each class."""
    iou_per_class = []
    for class_id in range(num_classes):
        # Create binary masks for the current class in both ground truth and prediction
        gt_class_mask = (gt_mask == class_id)
        pred_class_mask = (pred_mask == class_id)
        
        # Calculate intersection and union
        intersection = np.logical_and(gt_class_mask, pred_class_mask).sum()
        union = np.logical_or(gt_class_mask, pred_class_mask).sum()

        # Avoid division by zero
        if union == 0:
            iou = 1.0  # If there are no ground truth or predicted pixels, IoU is perfect
        else:
            iou = intersection / union
        
        iou_per_class.append(iou)

    return iou_per_class
def calculate_f1(precision, recall):
    return 2 * (precision * recall) / (precision + recall + 1e-7)
def calculate_accuracy(gt, pred):
    correct_predictions = tf.reduce_sum(tf.cast(tf.equal(gt, pred), dtype=tf.float32))
    total_pixels = tf.cast(tf.size(gt), dtype=tf.float32)
    return correct_predictions / total_pixels
def calculate_recall(gt, pred):
    # Cast tensors to float32
    gt = tf.cast(gt, tf.float32)
    pred = tf.cast(pred, tf.float32)
    
    true_positive = tf.reduce_sum(gt * pred)
    false_negative = tf.reduce_sum(gt) - true_positive
    return true_positive / (true_positive + false_negative + 1e-7)
def calculate_precision(gt, pred):
    # Cast tensors to float32
    gt = tf.cast(gt, tf.float32)
    pred = tf.cast(pred, tf.float32)
    
    true_positive = tf.reduce_sum(gt * pred)
    false_positive = tf.reduce_sum(pred) - true_positive
    return true_positive / (true_positive + false_positive + 1e-7)
def calculate_dice(gt, pred):
    # Cast the tensors to float32
    gt = tf.cast(gt, tf.float32)
    pred = tf.cast(pred, tf.float32)
    
    # Calculate the intersection and Dice coefficient
    intersection = tf.reduce_sum(gt * pred)
    return (2. * intersection) / (tf.reduce_sum(gt) + tf.reduce_sum(pred) + 1e-7)

def calculate_f1_per_class(precision, recall, num_classes):
    f1_per_class = []
    for class_id in range(num_classes):
        f1 = 2 * (precision[class_id] * recall[class_id]) / (precision[class_id] + recall[class_id] + 1e-7)
        f1_per_class.append(f1)
    return f1_per_class
def calculate_accuracy_per_class(gt_mask, pred_mask, num_classes):
    accuracy_per_class = []
    for class_id in range(num_classes):
        gt_class_mask = (gt_mask == class_id)
        pred_class_mask = (pred_mask == class_id)
        
        correct_predictions = tf.reduce_sum(tf.cast(tf.equal(gt_class_mask, pred_class_mask), dtype=tf.float32))
        total_pixels = tf.reduce_sum(tf.cast(gt_class_mask, tf.float32))
        
        if total_pixels == 0:
            accuracy_per_class.append(1.0)  # If no pixels of this class, assume perfect accuracy
        else:
            accuracy_per_class.append(correct_predictions / total_pixels)
    
    return accuracy_per_class
def calculate_recall_per_class(gt_mask, pred_mask, num_classes):
    recall_per_class = []
    for class_id in range(num_classes):
        gt_class_mask = tf.cast(gt_mask == class_id, tf.float32)
        pred_class_mask = tf.cast(pred_mask == class_id, tf.float32)
        
        true_positive = tf.reduce_sum(gt_class_mask * pred_class_mask)
        false_negative = tf.reduce_sum(gt_class_mask) - true_positive
        
        if (true_positive + false_negative) == 0:
            recall_per_class.append(1.0)
        else:
            recall_per_class.append(true_positive / (true_positive + false_negative + 1e-7))
    
    return recall_per_class
def calculate_precision_per_class(gt_mask, pred_mask, num_classes):
    precision_per_class = []
    for class_id in range(num_classes):
        gt_class_mask = tf.cast(gt_mask == class_id, tf.float32)
        pred_class_mask = tf.cast(pred_mask == class_id, tf.float32)
        
        true_positive = tf.reduce_sum(gt_class_mask * pred_class_mask)
        false_positive = tf.reduce_sum(pred_class_mask) - true_positive
        
        if (true_positive + false_positive) == 0:
            precision_per_class.append(1.0)
        else:
            precision_per_class.append(true_positive / (true_positive + false_positive + 1e-7))
    
    return precision_per_class
def calculate_dice_per_class(gt_mask, pred_mask, num_classes):
    dice_per_class = []
    for class_id in range(num_classes):
        gt_class_mask = tf.cast(gt_mask == class_id, tf.float32)
        pred_class_mask = tf.cast(pred_mask == class_id, tf.float32)
        
        intersection = tf.reduce_sum(gt_class_mask * pred_class_mask)
        dice_score = (2. * intersection) / (tf.reduce_sum(gt_class_mask) + tf.reduce_sum(pred_class_mask) + 1e-7)
        
        dice_per_class.append(dice_score)
    
    return dice_per_class

def display(display_list,label="output plots"):
    plt.figure(figsize=(15, 15))
    print(label)
    title = ["Input Image", "True Mask", "Predicted Mask","diff"]
    for i in range(len(display_list)):
        plt.subplot(1, len(display_list), i + 1)
        plt.title(title[i])
        # print(display_list[i].shape)
        img=tf.keras.utils.array_to_img(display_list[i])
        img_raw=display_list[i].numpy().astype(int)[:,:,0]
        if i>0:
            print(np.unique(img_raw))
            colored_mask = np.zeros_like(tf.keras.utils.array_to_img(display_list[0]))
            colored_mask[:, :, 0] = np.vectorize(map_to_b)(img_raw)
            colored_mask[:, :, 1] = np.vectorize(map_to_g)(img_raw)
            colored_mask[:, :, 2] = np.vectorize(map_to_r)(img_raw)
            img=colored_mask
        plt.imshow(img)
        plt.axis("off")
    plt.show()

def create_mask(pred_mask):
    pred_mask = tf.math.argmax(pred_mask, axis=1)
    pred_mask = tf.expand_dims(pred_mask, -1)
    return pred_mask
    
def cast_to_int32(image):
    # Normalize the image pixel values to [0, 1]
    image = tf.cast(image, tf.float32) / 255.0
    # Convert to uint8 by scaling the values back to [0, 255]
    image = tf.cast(image * 255, tf.int32)
    return image
    

def show_predictions(dataset=None, num=1):
    if dataset:
        for sample in dataset.take(num):
            
            images, masks = sample["pixel_values"], sample["labels"]
            print(images.shape)
            masks = tf.expand_dims(masks, -1)
            pred_masks = model.predict(images).logits
            images = tf.transpose(images, (0, 2, 3, 1))
            created_masks=cast_to_int32(create_mask(pred_masks))
            for i in range(0,len(created_masks)):
                gt=cast_to_int32(masks[i])
                created_mask=created_masks[i]
                created_mask = tf.image.resize(created_mask, [374, 1238],
                            method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
                

                diff_mask_road=abs(created_mask-gt)
                print(f"MSE: {calculate_mse(gt, created_mask)}")
                print(f"IOU: {calculate_iou(gt, created_mask,22)}")
                display([images[i], gt,created_mask,diff_mask_road ])




def calculate_tp_fp_tn_fn(gt_flat, pred_flat, num_classes):
    TP = np.zeros(num_classes)
    FP = np.zeros(num_classes)
    TN = np.zeros(num_classes)
    FN = np.zeros(num_classes)

    for class_idx in range(num_classes):
        gt_class = (gt_flat == class_idx)
        pred_class = (pred_flat == class_idx)
        
        TP[class_idx] += np.sum(np.logical_and(gt_class, pred_class))
        FP[class_idx] += np.sum(np.logical_and(~gt_class, pred_class))
        FN[class_idx] += np.sum(np.logical_and(gt_class, ~pred_class))
        TN[class_idx] += np.sum(np.logical_and(~gt_class, ~pred_class))
        
    return TP, FP, TN, FN

class DisplayCallback(tf.keras.callbacks.Callback):
    def __init__(self, dataset, **kwargs):
        super().__init__(**kwargs)
        self.dataset = dataset

    def on_epoch_end(self, epoch, logs=None):
        clear_output(wait=True)
        show_predictions(self.dataset)
        print("\nSample Prediction after epoch {}\n".format(epoch + 1))

def calculate_psnr(gt, pred, epsilon=1e-7):
    mse = np.mean((gt - pred) ** 2)
    if mse == 0:
        return np.inf
    max_pixel = np.max(gt)
    psnr = 20 * np.log10(max_pixel / (np.sqrt(mse) + epsilon))
    return psnr


def calculate_mae(gt, pred):
    return tf.reduce_mean(tf.abs(tf.cast(gt, tf.float32) - tf.cast(pred, tf.float32))).numpy()

def calculate_ssim(gt, pred):
    gt_img = gt.numpy().squeeze()
    pred_img = pred.numpy().squeeze()
    
    # Ensure images are at least 1x1
    if gt_img.size == 0 or pred_img.size == 0:
        return 0  # or handle accordingly
    
    # Ensure images have at least two dimensions
    if gt_img.ndim < 2 or pred_img.ndim < 2:
        raise ValueError("Images must have at least two dimensions")
    
    # Determine the minimum dimension size
    min_dim = min(gt_img.shape[0], gt_img.shape[1])
    
    # Set win_size appropriately
    win_size = min(7, min_dim)
    if win_size % 2 == 0:
        win_size = max(win_size - 1, 1)  # Ensure win_size is at least 1
    
    # Handle multichannel images
    if gt_img.ndim == 3:
        channel_axis = -1  # Assuming channels are in the last axis
    else:
        channel_axis = None  # Grayscale images
    
    # Compute SSIM
    ssim_value = ssim(
        gt_img,
        pred_img,
        data_range=gt_img.max() - gt_img.min(),
        win_size=win_size,
        channel_axis=channel_axis
    )
    return ssim_value
def show_metrics_extended_pert(dataset=None, num_classes=22, model=None, output_dir='metrics_logs', diff_images_dir='segmentation_differences', has_weather=True,pert="na"):
    import os
    import pandas as pd
    from PIL import Image
    import numpy as np
    import tensorflow as tf

    epsilon = 1e-7  # Assign at the beginning

    if dataset:
        total_samples = 0

        # Initialize accumulators per weather type
        weather_metrics = {}

        # Create directories if they don't exist
        # if not os.path.exists(diff_images_dir):
        #     os.makedirs(diff_images_dir)
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
        sample_index = 0  # To name the saved images uniquely

        for batch in dataset:
            if has_weather:
                images, masks, weather = batch["pixel_values"], batch["labels"], batch["weather"]
                unique_weathers = np.unique(weather)

                for w in unique_weathers:
                    # Filter images and masks by the current weather type
                    mask_for_weather = weather == w
                    images_weather = images[mask_for_weather]
                    masks_weather = masks[mask_for_weather]

                    if w not in weather_metrics:
                        weather_metrics[w] = {
                            'TP_total': np.zeros(num_classes),
                            'FP_total': np.zeros(num_classes),
                            'TN_total': np.zeros(num_classes),
                            'FN_total': np.zeros(num_classes),
                            'total_mse': 0,
                            'total_mae': 0,
                            'total_pixel_accuracy': 0,
                            'total_pixels_considered': 0,
                            'per_class_metrics': {class_idx: {'TP': 0, 'FP': 0, 'FN': 0, 'TN': 0} for class_idx in range(num_classes)}
                        }

                    # Process the batch for this specific weather
                    masks_weather = tf.expand_dims(masks_weather, -1)
                    pred_logits = model.predict(images_weather).logits
                    images_weather = tf.transpose(images_weather, (0, 2, 3, 1))
                    pred_masks = create_mask(pred_logits)

                    gt = cast_to_int32(masks_weather)
                    pred = cast_to_int32(pred_masks)

                    pred = tf.image.resize(pred, [374, 1238], method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
                    gt = tf.image.resize(gt, [374, 1238], method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)

                    gt_flat = tf.reshape(gt, [-1]).numpy()
                    pred_flat = tf.reshape(pred, [-1]).numpy()

                    valid_mask = gt_flat != 255
                    gt_filtered = gt_flat[valid_mask]
                    pred_filtered = pred_flat[valid_mask]

                    mse_value = np.mean((gt_filtered - pred_filtered) ** 2)
                    mae_value = np.mean(np.abs(gt_filtered - pred_filtered))

                    weather_metrics[w]['total_mse'] += mse_value * len(gt_filtered)
                    weather_metrics[w]['total_mae'] += mae_value * len(gt_filtered)
                    weather_metrics[w]['total_pixels_considered'] += len(gt_filtered)

                    TP, FP, TN, FN = calculate_tp_fp_tn_fn(gt_filtered, pred_filtered, num_classes)
                    weather_metrics[w]['TP_total'] += TP
                    weather_metrics[w]['FP_total'] += FP
                    weather_metrics[w]['TN_total'] += TN
                    weather_metrics[w]['FN_total'] += FN

                    correct_pixels = np.sum(gt_filtered == pred_filtered)
                    pixel_accuracy = correct_pixels / (len(gt_filtered) + epsilon)
                    weather_metrics[w]['total_pixel_accuracy'] += pixel_accuracy * len(gt_filtered)

                    # Accumulate per-class metrics
                    for class_idx in range(num_classes):
                        weather_metrics[w]['per_class_metrics'][class_idx]['TP'] += TP[class_idx]
                        weather_metrics[w]['per_class_metrics'][class_idx]['FP'] += FP[class_idx]
                        weather_metrics[w]['per_class_metrics'][class_idx]['FN'] += FN[class_idx]
                        weather_metrics[w]['per_class_metrics'][class_idx]['TN'] += TN[class_idx]

                    # Save difference images (for valid pixels)
                    difference = np.abs(gt_filtered - pred_filtered)
                    difference_image = np.zeros_like(gt_flat)
                    difference_image[valid_mask] = difference
                    difference_image = difference_image.reshape(gt.shape[0], gt.shape[1], gt.shape[2])
                    difference_image = difference_image.astype(np.uint8) * (255 // num_classes)
                    char = "'"
                    weather_string = str(w).split(char)[1]
                    # if not os.path.exists(os.path.join(diff_images_dir, weather_string)):
                    #     os.makedirs(os.path.join(diff_images_dir, weather_string))
                    for i in range(difference_image.shape[0]):
                        # image_path = os.path.join(diff_images_dir, f'{weather_string}/{sample_index}.png')
                        # Image.fromarray(difference_image[i].squeeze(), mode='L').save(image_path)
                        sample_index += 1

                    total_samples += gt.shape[0]

                    if total_samples % 100 == 0:
                        print(f"Processed {total_samples} samples")

        # After processing the batches, calculate metrics for each weather type
        overall_weather_metrics = {}
        per_class_weather_metrics = {}

        for w, metrics in weather_metrics.items():
            TP_total = metrics['TP_total']
            FP_total = metrics['FP_total']
            TN_total = metrics['TN_total']
            FN_total = metrics['FN_total']

            precision = TP_total / (TP_total + FP_total + epsilon)
            recall = TP_total / (TP_total + FN_total + epsilon)
            f1_score = 2 * precision * recall / (precision + recall + epsilon)
            accuracy = (TP_total + TN_total) / (TP_total + FP_total + TN_total + FN_total + epsilon)
            iou = TP_total / (TP_total + FP_total + FN_total + epsilon)
            dice = 2 * TP_total / (2 * TP_total + FP_total + FN_total + epsilon)
            specificity = TN_total / (TN_total + FP_total + epsilon)

            avg_mse = metrics['total_mse'] / (metrics['total_pixels_considered'] + epsilon)
            avg_mae = metrics['total_mae'] / (metrics['total_pixels_considered'] + epsilon)
            avg_pixel_accuracy = metrics['total_pixel_accuracy'] / (metrics['total_pixels_considered'] + epsilon)

            relevant_TP = TP_total.sum()
            relevant_FP = FP_total.sum()
            relevant_TN = TN_total.sum()
            relevant_FN = FN_total.sum()

            overall_precision = relevant_TP / (relevant_TP + relevant_FP + epsilon)
            overall_recall = relevant_TP / (relevant_TP + relevant_FN + epsilon)
            overall_specificity = relevant_TN / (relevant_TN + relevant_FP + epsilon)
            overall_accuracy = (relevant_TP + relevant_TN) / (relevant_TP + relevant_FP + relevant_FN + relevant_TN + epsilon)
            overall_f1_score = 2 * overall_precision * overall_recall / (overall_precision + overall_recall + epsilon)
            overall_iou = relevant_TP / (relevant_TP + relevant_FP + relevant_FN + epsilon)
            overall_dice = 2 * relevant_TP / (2 * relevant_TP + relevant_FP + relevant_FN + epsilon)

            # Store overall metrics for weather
            overall_weather_metrics[w] = {
                'Weather Type': w,
                'Average MSE': avg_mse,
                'Average MAE': avg_mae,
                'Average Pixel Accuracy': avg_pixel_accuracy,
                'Overall Precision': overall_precision,
                'Overall Recall': overall_recall,
                'Overall Specificity': overall_specificity,
                'Overall F1 Score': overall_f1_score,
                'Overall Accuracy': overall_accuracy,
                'Overall IoU': overall_iou,
                'Overall Dice Coefficient': overall_dice,
            }

            # Calculate per-class metrics for each weather type
            per_class_metrics = []
            for class_idx in range(num_classes):
                per_class_TP = metrics['per_class_metrics'][class_idx]['TP']
                per_class_FP = metrics['per_class_metrics'][class_idx]['FP']
                per_class_FN = metrics['per_class_metrics'][class_idx]['FN']
                per_class_TN = metrics['per_class_metrics'][class_idx]['TN']

                precision_class = per_class_TP / (per_class_TP + per_class_FP + epsilon)
                recall_class = per_class_TP / (per_class_TP + per_class_FN + epsilon)
                f1_class = 2 * precision_class * recall_class / (precision_class + recall_class + epsilon)
                iou_class = per_class_TP / (per_class_TP + per_class_FP + per_class_FN + epsilon)
                dice_class = 2 * per_class_TP / (2 * per_class_TP + per_class_FP + per_class_FN + epsilon)

                per_class_metrics.append({
                    'Class': class_idx,
                    'Precision': precision_class,
                    'Recall': recall_class,
                    'F1 Score': f1_class,
                    'IoU': iou_class,
                    'Dice Coefficient': dice_class
                })

            # Store per-class metrics for this weather type
            per_class_weather_metrics[w] = pd.DataFrame(per_class_metrics)

        # Convert to DataFrame and save to CSV
        overall_weather_metrics_df = pd.DataFrame(overall_weather_metrics).T
        overall_weather_metrics_df.to_csv(os.path.join(output_dir, f'overall_pert_metrics_({pert}).csv'), index=False)

        for w, per_class_df in per_class_weather_metrics.items():
            per_class_df.to_csv(os.path.join(output_dir, f'per_class_metrics_({w}).csv'), index=False)

        print("\nOverall Metrics by Weather Type:")
        print(overall_weather_metrics_df)


In [ ]:
from transformers import  TFSegformerForSemanticSegmentation
from PIL import Image
import requests

model_checkpoint = "nvidia/segformer-b0-finetuned-cityscapes-640-1280"

# id2label mapping for the KITTI dataset
label2id = {
        'Road': 0,
        'Sidewalk': 1,
        'Building': 2,
        'Wall': 3,
        'Fence': 4,
        'Pole': 5,
        'TrafficLight': 6,
        'TrafficSign': 7,
        'Vegetation': 8,
        'Terrain': 9,
        'Sky': 10,
        'Person': 11,
        'Rider': 12,
        'Car': 13,
        'Truck': 14,
        'Bus': 15,
        'Train': 16,
        'Motorcycle': 17,
        'Bicycle': 18,
        'Void': 255,
        'Misc': 19,
        'GuardRail': 20,
    }
id2label = {label: id for id, label in label2id.items()}
num_labels = len(id2label)
print(len(id2label))
model = TFSegformerForSemanticSegmentation.from_pretrained(
    model_checkpoint,
    num_labels=num_labels,
    id2label=id2label,
    label2id=label2id,
    ignore_mismatched_sizes=True,
)

lr = 0.00006
optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
model.compile(optimizer=optimizer)
checkpoint_file_path = "./content/Model_original.hdf5"
model.load_weights(checkpoint_file_path)

In [ ]:
perturbations=['canny_edges_mapping','contrast','cutout_filter','defocus_blur','dotted_lines_mapping','elastic','false_color_filter','fog_filter','frost_filter','gaussian_blur','gaussian_noise','glass_blur','grayscale_filter','histogram_equalisation','impulse_noise','increase_brightness','jpeg_filter','low_pass_filter','motion_blur','phase_scrambling','poisson_noise','reflection_filter','sample_pairing_filter','saturation_decrease_filter','saturation_filter','scale_image','sharpen_filter','snow_filter','speckle_noise_filter','splatter_mapping','translate_image','white_balance_filter','zigzag_mapping']
for perturbation in perturbations:
    loaded_dictionary_images_sim={}
    loaded_semantic_id_sim={}
    file_path = f'/Volumes/Volume/new_perts/({perturbation})_raw_image_sim.h5'
    loaded_dictionary_images_sim[perturbation]=load_h5_to_dictionary(file_path)
    file_path = f'/Volumes/Volume/new_perts/({perturbation})_semantic_id_list_sim.h5'
    loaded_semantic_id_sim[perturbation]=load_h5_to_dictionary(file_path)
    internal_test_index_list=list(loaded_dictionary_images_sim[perturbation])
    print(internal_test_index_list)
    lowest_height=374
    lowest_width=1238 
    for dataset_index in internal_test_index_list:
        loaded_dictionary_images_sim[perturbation][dataset_index] = crop_images_to_lowest_dimensions(loaded_dictionary_images_sim[perturbation][dataset_index], lowest_height, lowest_width)
        loaded_semantic_id_sim[perturbation][dataset_index] = crop_1d_to_lowest_dimensions(loaded_semantic_id_sim[perturbation][dataset_index], lowest_height, lowest_width)

    test_indexes_sim = {}
    test_indexes_weather = {}
    indexes_real = {}
    train_indexes_sim_perturbed = {}
    valid_indexes_sim_perturbed = {}
    test_dataset_inner = {}
    for dataset_index in internal_test_index_list:
      test_dataset_inner[dataset_index]=[]
      for index in range(1, len(loaded_dictionary_images_sim[perturbation][dataset_index])):
        test_dataset_inner[dataset_index].append(index)
    test_indexes_sim[perturbation]=test_dataset_inner
    print("here")
    print(internal_test_index_list)
    for dataset_index in internal_test_index_list:
        print("Dataset", perturbation,dataset_index)
        print("Test sim: ",len(test_indexes_sim[perturbation][dataset_index]))

    tf_dataset_test_sim = create_tf_dataset_pert(loaded_dictionary_images_sim, loaded_semantic_id_sim,test_indexes_sim,[perturbation])
    test_dataset =tf_dataset_test_sim
    auto = tf.data.AUTOTUNE


    test_ds = (
        test_dataset
        .prefetch(auto)
    )
    show_metrics_extended_pert(test_ds,num_classes=22, model=model, output_dir='./content/evaluation/model_pert_perclass', diff_images_dir='./content/evaluation/model_pert_segmentations_perclass',has_weather=True,pert=perturbation)
